# Purpose
This notebook demonstrates validation and invalidation of a Brick v1.2.0 model. 
The data graph is instantiated using [oomason](https://github.com/gtfierro/oomason), which has not been incorporated into `tasty` yet.  
All of the following 13 shapes from Brick v1.2.0 are supported with the goal of supporting new shapes as they are added to future releases of Brick. 
- [x] AggregationShape 
- [x] AreaShape
<!-- - [ ] *AzimuthShape -->
- [x] BuildingPrimaryFunctionShape
- [x] CoolingCapacityShape
<!-- - [ ] CoordinateShape -->
- [x] CurrentFlowTypeShape
<!-- - [ ] *EfficiencyShape -->
<!-- - EquipmentShape -->
<!-- - LocationShape -->
- [x] PhaseCountShape
- [x] PhasesShape
<!-- - PointShape -->
- [x] PowerComplexityShape
- [x] PowerFlowShape
<!-- - [ ] *PowerOutputShape -->
- [x] StageShape
<!-- - [ ] *TemperatureCoefficientPerDegreeCelsiusShape -->
<!-- - [ ] *TemperatureShape -->
- [x] ThermalTransmittanceShape
<!-- - [ ] *TiltShape -->
<!-- - TimeseriesReference -->
- [x] VolumeShape
- [x] YearBuiltShape

<!-- *added to Brick after v1.2.0 -->


In [1]:
from pyshacl import validate
from rdflib import Namespace, Literal, XSD

from mason import BrickClassGenerator, compile_model, Brick

import tasty.constants as tc
import tasty.graphs as tg
from tasty.shapes_loader import ShapesLoader

# Setup

In [2]:
# pass or fail setting
fail = False

# print graphs setting
print_graphs = True

# namespace
EX = Namespace('urn:example#')

# entity propterties

fail_str = '_fail'
int_num = 2021
flt_num = 2021.0
degrees = 20.21
percent = 0.2021

# building
year_built = int_num
primary_function = 'Adult Education'
thermal_transmittance = flt_num

# room
area = flt_num
volume = flt_num

# chiller
cooling_capacity = flt_num
stages = 2

# PV panel electrical
current_flow_type = 'AC'
phase_count = '2'
phases = 'A' 
power_complexity = 'real'
power_flow = 'import'

# point
aggregation = 'max'

if fail:

    # building
    year_built = str(year_built)
    primary_function = primary_function + fail_str
    thermal_transmittance = str(thermal_transmittance)
 
    # room
    area = str(area)
    volume = str(volume)

    # chiller
    
    cooling_capacity = str(cooling_capacity)
    stages = str(stages)

    # PV electrical
    current_flow_type = current_flow_type + fail_str 
    phase_count = phase_count + fail_str 
    phases = phases + fail_str 
    power_complexity = power_complexity + fail_str 
    power_flow = power_flow + fail_str

    # point
    aggregation = aggregation + fail_str   


# Data Graph (oomason)

## Shapes

In [3]:
# building
year_built_shape = Brick.EntityProperty.YearBuiltShape(year_built)
primary_function_shape = Brick.EntityProperty.BuildingPrimaryFunctionShape(Literal(primary_function, datatype=XSD.string)) # newer RDFlib is more specific about some datatypes
thermal_transmittance_shape = Brick.EntityProperty.ThermalTransmittenceShape(thermal_transmittance, Brick.Unit.Watt_per_Square_Meter_Kelvin) # transmittence misspelled, check IP units

# room
area_shape = Brick.EntityProperty.AreaShape(area, Brick.Unit.Square_Foot)
volume_shape = Brick.EntityProperty.VolumeShape(volume, Brick.Unit.Cubic_Foot)

# chiller
cooling_capacity_shape = Brick.EntityProperty.CoolingCapacityShape(cooling_capacity, Brick.Unit.Watt)
stage_shape = Brick.EntityProperty.StageShape(stages)

# pv panel electrical
current_flow_type_shape = Brick.EntityProperty.CurrentFlowTypeShape(Literal(current_flow_type, datatype=XSD.string))
phases_shape = Brick.EntityProperty.PhasesShape(Literal(phases, datatype=XSD.string))
phase_count_shape = Brick.EntityProperty.PhaseCountShape(Literal(phase_count, datatype=XSD.string))
power_complexity_shape = Brick.EntityProperty.PowerComplexityShape(Literal(power_complexity, datatype=XSD.string))
power_flow_shape = Brick.EntityProperty.PowerFlowShape(Literal(power_flow, datatype=XSD.string))

# point
aggregation_shape = Brick.EntityProperty.AggregationShape(Literal(aggregation, datatype=XSD.string), Literal('max'))


## Entities

### Building

In [4]:
# entity
building = Brick.Building(EX["building"], "Building")

# add shapes
building.add_yearBuilt(year_built_shape)
building.add_buildingPrimaryFunction(primary_function_shape)
building.add_thermalTransmittence(thermal_transmittance_shape)

### Room

In [5]:
# entity
room = Brick.Room(EX['room1'], 'Room 1')

# add shapes
room.add_area(area_shape)
room.add_volume(volume_shape)

### Chiller

In [6]:
# entitiy
chiller = Brick.Chiller(EX['ch-1'], 'CH-1')

# add shapes
chiller.add_coolingCapacity(cooling_capacity_shape)
chiller.add_operationalStage(stage_shape)

### PV Panel

In [7]:
# entity
pv_panel = Brick.PV_Panel(EX['panel-1'], 'Panel-1')

# add shapes
pv_panel.add_currentFlowType(current_flow_type_shape)
pv_panel.add_electricalPhases(phases_shape)
pv_panel.add_electricalPhaseCount(phase_count_shape)
pv_panel.add_powerComplexity(power_complexity_shape)
pv_panel.add_powerFlow(power_flow_shape)

### Point

In [8]:
# entity
active_power_sensor = Brick.Active_Power_Sensor(EX['active-power-sensor-1'], 'Active Power Sensor 1')

# add shapes
active_power_sensor.add_aggregate(aggregation_shape)

In [9]:
# compile data graph, which validates it too
data_graph = compile_model([('ex', EX)]) # provide namespaces when compiling

# print
if print_graphs: tg.print_graph(data_graph)

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix ex: <urn:example#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:active-power-sensor-1 a brick:Active_Power_Sensor ;
    brick:aggregate [ a brick:AggregationShape ;
            brick:aggregationFunction "max" ;
            brick:aggregationInterval "max"^^xsd:string ] .

ex:building a brick:Building ;
    brick:buildingPrimaryFunction [ a brick:BuildingPrimaryFunctionShape ;
            brick:value "Adult Education"^^xsd:string ] ;
    brick:thermalTransmittence [ a brick:ThermalTransmittenceShape ;
            brick:hasUnit unit:W-PER-M2-K ;
            brick:value 2.021e+03 ] ;
    brick:yearBuilt [ a brick:YearBuiltShape ;
            brick:value 2021 ] .

ex:ch-1 a brick:Chiller ;
    brick:coolingCapacity [ a brick:CoolingCapacityShape ;
            brick:hasUnit unit:W ;
            brick:value 2.021e+03 ] ;
    brick:operationalStage [ a brick:StageShape ;
 

# Shapes Graph (tasty)

In [10]:
schema = tc.BRICK
version = tc.V1_2

# load ontology
brick_ontology = tg.load_ontology(schema, version)

# shapes graph
sl = ShapesLoader(schema)
shapes_graph = sl.load_all_shapes()
shapes_graph.bind('ex', EX)

# print
if print_graphs: tg.print_graph(shapes_graph)

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

brick:AggregationShape a owl:Class,
        sh:NodeShape ;
    sh:property [ a sh:PropertyShape ;
            skos:definition "Interval expressed in an ISO 8601 Duration string, e.g. RP1D" ;
            sh:datatype xsd:string ;
            sh:minCount 1 ;
            sh:path brick:aggregationInterval ],
        [ a sh:PropertyShape ;
            skos:definition "The aggregation function applied to data in the interval which produces the value" ;
            sh:in ( "max" "min" "count" "mean" "sum" "median" "mode" ) ;
            sh:minCount 1 ;
            sh:path brick:aggregationFunction ] .

brick:AreaShape a owl:Class,
      

# Validate (RDFlib)

In [11]:
conforms, results_graph, results_text = validate(data_graph=data_graph, shacl_graph=shapes_graph, ont_graph=brick_ontology)
print(results_text)

Validation Report
Conforms: True

